In [111]:
import openpyxl as opl
#import related to tkinter file dialog
from tkinter import *
from tkinter import ttk
from tkinter import filedialog as fd
import pandas as pd
import os
import math
import datetime
import shutil
import numpy as np
#logg;ing related info
import logging
import xlsxwriter
import pandas as pd
from scipy import interpolate

In [112]:
"""This function calculate_return_period_correction_factor """
def calculate_return_period_correction_factor(T,rt_df):
    """
    parameter description:
    T: return period for which correction shall be calcualted
    rt_df:table of correction    
    """
    x=rt_df['Return_Period']
    y=rt_df['Multyplying _Factor']
    f1=interpolate.interp1d(x,y)
    mf=f1(T)
    return mf
    pass

In [113]:
"""This function calculate Basin Characteristics Correction Factor"""
def calculate_basin_characteristics_correction_factor(sf,adjustment_df):
    """
        parameter description:
        sf: [(L**2)/s]^0.3
        rt_df:table of correction    
    """
    #Slope_Factor	Adjustment
    if sf>7:
        cf=-40
    else:
        
        x=adjustment_df['Slope_Factor']
        y=adjustment_df['Adjustment']
        f2=interpolate.interp1d(x,y)
        cf=f2(sf)
    return cf   

In [114]:
"""This function calculate Basin Size Correction Factors"""
def calculate_basin_size_correction_factor(basin_area):
    if basin_area>40:
        cf=20
    else:
        cf=70-1.25*basin_area
    return cf
    pass


In [115]:
"""This Function Converts MKS input data to FPS """
def covert_main_df_to_fps(input_df):
    output_df=input_df.copy(deep=True)
    output_df["Basin_Area"]=output_df["Basin_Area"]/(1.609*1.609)
    #Length of Khal	Slope
    output_df["Length_of_Khal"]=output_df["Length_of_Khal"]/(1.609)
    output_df["Slope"]=output_df["Slope"]/(1000*100)
    return output_df

In [116]:
"""This Function Converts MKS input data to FPS """
def calcualte_design_discharge():
    pass

In [117]:
"""This Files Saves a list of Data Frame"""
def saveDataFrame(dataframes,filepath,names):
    #writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    #writer= pd.ExcelWriter(filepath)
    for df,sname in zip(dataframes,names):
        df.to_excel(filepath,sheet_name=sname)   
    #writer.save()  
"""This Files Writes Data Frame in Proper Place of a Excel Sheet"""
"""Functions for Transferring Data Frame to Autocad Input File"""
def writeDataFrameToExcel(data_frame,drow,dcol,wbook_name,wsheet_name):
    """
    Parameters:
    data_frame= name of the frame from which data to be transferred
    drow=row offset to be added to dframe row index
    dcol=column offset to be added to dframe column index
    wbook_name=name of the destination work book
    wsheet_name=name of the destination work sheet    
    """
    wb=opl.load_workbook(wbook_name)
    #print(wb.sheetnames)
    transfer_sheet=wb[wsheet_name]
    dfshape=data_frame.shape 
    print(dfshape)
    for i in range(0,dfshape[0]):
        row_index=i+drow
        for j in range(0,dfshape[1]):
            transfer_sheet.cell(row=row_index,column=j+dcol).value=data_frame.iloc[i,j]
    wb.save(wbook_name)

In [118]:
"""
#############################################################################################################################
Programme Execution Begins Here 
#############################################################################################################################

"""
#create an instance of tkinter frame or windowroot=Tk()
#create an instance of tkinter frame or window
root=Tk()
root.geometry("750x150")
work_book_path=fd.askopenfilename(title="select Excel Input File")
root.withdraw()
#print(work_book_path)
dir_name=os.path.dirname(work_book_path)
head,tail=os.path.split(work_book_path)
print(dir_name)
print(tail)
file_names=tail.split(".")
print(dir_name)
out_path=os.path.join(dir_name,"Hydrologic_Design_"+file_names[0]+"."+file_names[1])
out_path2=os.path.join(dir_name,"data_agg_output.xlsx")

E:/Design_All_DC_5/AEMostofa/Southwest Project/SMO Gopalgonj/Khal/P1/XL DATA
Hydrologic Design Input.xlsx
E:/Design_All_DC_5/AEMostofa/Southwest Project/SMO Gopalgonj/Khal/P1/XL DATA


In [119]:
#inthis cell wefind direction of datamatrix

In [120]:
#settingpup log files
log_file_path=os.path.join(dir_name,"data_agg.log")
logger=logging.getLogger(__name__)
logger.setLevel("DEBUG")
fileHandler=logging.FileHandler(filename=log_file_path,mode="w")
formatter=logging.Formatter('%(asctime)s - %(name)s - %(lineno)s - %(levelname)s - %(message)s')
fileHandler.setFormatter(formatter)
fileHandler.setLevel(logging.DEBUG)
logger.addHandler(fileHandler)
#formatter=logging.Formatter("{asctime}-{levelname}-{message}", style="{",datefmt="%Y-%m-%d %H:%M %S")
#testing logger Setup........................
logger.debug("Sucessfully Setup Logger for this programme.............")
#creating a second log file path and handler for handling individual function
log_file_path_function=os.path.join(dir_name,"function_log.log")
fileHandler_function=logging.FileHandler(filename=log_file_path_function,mode="w")
fileHandler_function.setFormatter(formatter)
fileHandler_function.setLevel(logging.DEBUG)

                                    

In [121]:
#work_book_path
shheetName="Main_Input"
#sheet_name
Main_Input_Df_Mks=pd.read_excel(work_book_path,sheet_name=shheetName)
Main_Input_Df_Mks

,Code_Name,Khal_Name,Basin_Area,Length_of_Khal,Slope,Return_Period,q_100
0,HBK,Hiron Branch Khal,2,0.678,5,10,155
1,HMP,Hiron Middle Para Khal,2,0.714,5,10,155
2,BHK,Boroi Hati Khal,2,0.600,5,10,155
3,RKK,Rakhila Khal,2,1.768,5,10,155
4,GZK,Gazir Khal,3,2.325,5,10,155
5,SSK,Silna -Shir Khal,2,1.570,5,10,155
6,GBK,Gabir Beel Khal,3,2.170,5,10,155
7,KPK,Khanar Par Khal,3,2.330,5,10,155
8,DLB,Dola Beel,3,2.260,5,10,155
9,DLK,Dhulorujir Khal,2,0.950,5,10,155


In [122]:
shheetName="Return_Period_Correction"
Return_Period_Correction_Df=pd.read_excel(work_book_path,sheet_name=shheetName)
Return_Period_Correction_Df

,Return_Period,Multyplying _Factor
0,500,1.14
1,200,1.06
2,50,0.93
3,25,0.85
4,15,0.79
5,10,0.73
6,5,0.62
7,2,0.40


In [123]:
shheetName="Basin_Slope_Correction"
Basin_Slope_Correction_Df=pd.read_excel(work_book_path,sheet_name=shheetName)
Basin_Slope_Correction_Df

,Slope_Factor,Adjustment
0,0.98,40
1,2.00,10
2,3.00,-10
3,4.00,-22
4,5.00,-31
5,6.00,-38
6,7.00,40


In [124]:
Main_Input_Df_fps=covert_main_df_to_fps(Main_Input_Df_Mks)
Main_Input_Df_fps

,Code_Name,Khal_Name,Basin_Area,Length_of_Khal,Slope,Return_Period,q_100
0,HBK,Hiron Branch Khal,0.772535,0.421380,0.00005,10,155
1,HMP,Hiron Middle Para Khal,0.772535,0.443754,0.00005,10,155
2,BHK,Boroi Hati Khal,0.772535,0.372902,0.00005,10,155
3,RKK,Rakhila Khal,0.772535,1.098819,0.00005,10,155
4,GZK,Gazir Khal,1.158802,1.444997,0.00005,10,155
5,SSK,Silna -Shir Khal,0.772535,0.975761,0.00005,10,155
6,GBK,Gabir Beel Khal,1.158802,1.348664,0.00005,10,155
7,KPK,Khanar Par Khal,1.158802,1.448104,0.00005,10,155
8,DLB,Dola Beel,1.158802,1.404599,0.00005,10,155
9,DLK,Dhulorujir Khal,0.772535,0.590429,0.00005,10,155


In [125]:
mf=calculate_return_period_correction_factor(17,Return_Period_Correction_Df)
print(mf)

0.802


In [126]:
khal_name=Main_Input_Df_fps.iloc[11,1]
A=Main_Input_Df_fps.iloc[11,2]
L=Main_Input_Df_fps.iloc[11,3]
s=Main_Input_Df_fps.iloc[11,4]
T=Main_Input_Df_fps.iloc[11,5]
q_100=Main_Input_Df_fps.iloc[11,6]
print("khal name={} slope={} T={} q_100={}".format(khal_name,s,T,q_100))
mf=calculate_return_period_correction_factor(T,Return_Period_Correction_Df)
q_10=q_100*mf
slope_factor=math.pow((L**2)/s,0.3)
print("khal name={} q_100={} mf={} q_10={}".format(khal_name,q_100,mf,q_10))
print("slope factor={}".format(slope_factor))
##############################
cf=calculate_basin_characteristics_correction_factor(slope_factor,Basin_Slope_Correction_Df)
q_adjusted=q_10*(1+cf/100.0)
print("Khal name={} sf={} correction={} q_adjusted={}".format(khal_name,slope_factor,cf,q_adjusted))
bcf=calculate_basin_size_correction_factor(A)
q_corrected=q_adjusted*(1+bcf/100.0)
print("Khal name={} A={} bcf={} q_adjusted={} q_corrected={}".format(khal_name,A,bcf,q_adjusted,q_corrected))

khal name=Kochu Khal slope=5e-05 T=10 q_100=155
khal name=Kochu Khal q_100=155 mf=0.73 q_10=113.14999999999999
slope factor=24.240555065359917
Khal name=Kochu Khal sf=24.240555065359917 correction=-40 q_adjusted=67.88999999999999
Khal name=Kochu Khal A=1.1588018143746277 bcf=68.55149773203172 q_adjusted=67.88999999999999 q_corrected=114.4296118102763


In [127]:
khal_name=Main_Input_Df_fps.iloc[12,1]
A=Main_Input_Df_fps.iloc[12,2]
L=Main_Input_Df_fps.iloc[12,3]
s=Main_Input_Df_fps.iloc[12,4]
T=Main_Input_Df_fps.iloc[12,5]
q_100=Main_Input_Df_fps.iloc[12,6]
print("khal name={} slope={} T={} q_100={}".format(khal_name,s,T,q_100))
mf=calculate_return_period_correction_factor(T,Return_Period_Correction_Df)
q_10=q_100*mf
slope_factor=math.pow((L**2)/s,0.3)
print("khal name={} q_100={} mf={} q_10={}".format(khal_name,q_100,mf,q_10))
print("slope factor={}".format(slope_factor))
##############################
cf=calculate_basin_characteristics_correction_factor(slope_factor,Basin_Slope_Correction_Df)
q_adjusted=q_10*(1+cf/100.0)
print("Khal name={} sf={} correction={} q_adjusted={}".format(khal_name,slope_factor,cf,q_adjusted))
bcf=calculate_basin_size_correction_factor(A)
q_corrected=q_adjusted*(1+bcf/100.0)
print("Khal name={} A={} bcf={} q_adjusted={} q_corrected={}".format(khal_name,A,bcf,q_adjusted,q_corrected))

khal name=Aria Khal slope=5e-05 T=10 q_100=155
khal name=Aria Khal q_100=155 mf=0.73 q_10=113.14999999999999
slope factor=10.84997149054973
Khal name=Aria Khal sf=10.84997149054973 correction=-40 q_adjusted=67.88999999999999
Khal name=Aria Khal A=0.3862672714582092 bcf=69.51716591067724 q_adjusted=67.88999999999999 q_corrected=115.08520393675876


In [128]:
khal_name_list=[]
A_list=[]
L_list=[]
s_list=[]
T_list=[]
q_100_list=[]
mf_list=[]
q_10_list=[]
slope_factor_list=[]
cf_list=[]
q_adjusted_list=[]
bcf_list=[]
q_corrected_list=[]
Q_cusec_list=[]
Q_cumec_list=[]
Code_Name_list=[]
for index,row in Main_Input_Df_fps.iterrows():
    Code_Name=Main_Input_Df_fps.iloc[index,0]
    Code_Name_list.append(Code_Name)
    khal_name=Main_Input_Df_fps.iloc[index,1]    
    A=Main_Input_Df_fps.iloc[index,2]
    L=Main_Input_Df_fps.iloc[index,3]
    s=Main_Input_Df_fps.iloc[index,4]
    T=Main_Input_Df_fps.iloc[index,5]
    q_100=Main_Input_Df_fps.iloc[index,6] 
    #Code_Name	Khal_Name	Basin_Area	Length_of_Khal	Slope	Return Period	q_100
    khal_name_list.append(khal_name)
    A_list.append(A)
    L_list.append(L)
    s_list.append(s)
    T_list.append(T)
    
    """ 
    khal_name=row['Khal_Name'].iloc[0]    
    A=row['Basin_Area'].iloc
    L=Main_Input_Df_fps['Length_of_Khal']
    s=row['Slope']
    T=row['Return_Period']
    q_100=row['q_100'] 
    """
    print("khal name={} slope={} T={} q_100={}".format(khal_name,s,T,q_100))
    mf=calculate_return_period_correction_factor(T,Return_Period_Correction_Df)
    mf_list.append(mf)
    q_10=q_100*mf
    q_10_list.append(q_10)
    slope_factor=math.pow((L**2)/s,0.3)
    slope_factor_list.append(slope_factor)
    print("khal name={} q_100={} mf={} q_10={}".format(khal_name,q_100,mf,q_10))
    print("slope factor={}".format(slope_factor))
    ##############################
    cf=calculate_basin_characteristics_correction_factor(slope_factor,Basin_Slope_Correction_Df)
    cf_list.append(cf)
    q_adjusted=q_10*(1+cf/100.0)
    q_adjusted_list.append(q_adjusted)
    print("Khal name={} sf={} correction={} q_adjusted={}".format(khal_name,slope_factor,cf,q_adjusted))
    bcf=calculate_basin_size_correction_factor(A)
    bcf_list.append(bcf)
    q_corrected=q_adjusted*(1+bcf/100.0)
    q_corrected_list.append(q_corrected)
    Q_cusec= q_corrected*A
    Q_cumec=Q_cusec/35.314
    Q_cusec_list.append( Q_cusec)
    Q_cumec_list.append(Q_cumec)
    print("Khal name={} A={} bcf={} q_adjusted={} q_corrected={}".format(khal_name,A,bcf,q_adjusted,q_corrected))
    print("Khal name={} A={} q_corected={} Q_cusec={} Q_cumec={}".format(khal_name,A,q_corrected,Q_cusec,Q_cumec))
    pass


khal name=Hiron Branch Khal slope=5e-05 T=10 q_100=155
khal name=Hiron Branch Khal q_100=155 mf=0.73 q_10=113.14999999999999
slope factor=11.617513573344834
Khal name=Hiron Branch Khal sf=11.617513573344834 correction=-40 q_adjusted=67.88999999999999
Khal name=Hiron Branch Khal A=0.7725345429164184 bcf=69.03433182135447 q_adjusted=67.88999999999999 q_corrected=114.75740787351754
Khal name=Hiron Branch Khal A=0.7725345429164184 q_corected=114.75740787351754 Q_cusec=88.65406163784087 Q_cumec=2.510450859088205
khal name=Hiron Middle Para Khal slope=5e-05 T=10 q_100=155
khal name=Hiron Middle Para Khal q_100=155 mf=0.73 q_10=113.14999999999999
slope factor=11.983793016467931
Khal name=Hiron Middle Para Khal sf=11.983793016467931 correction=-40 q_adjusted=67.88999999999999
Khal name=Hiron Middle Para Khal A=0.7725345429164184 bcf=69.03433182135447 q_adjusted=67.88999999999999 q_corrected=114.75740787351754
Khal name=Hiron Middle Para Khal A=0.7725345429164184 q_corected=114.75740787351754 Q

In [130]:
khal_name_list=[]
A_list=[]
L_list=[]
s_list=[]
T_list=[]
q_100_list=[]
mf_list=[]
q_10_list=[]
slope_factor_list=[]
cf_list=[]
q_adjusted_list=[]
bcf_list=[]
q_corrected_list=[]
Q_cusec_list=[]
Q_cumec_list=[]
#Code_Name	Khal_Name	Basin_Area	Length_of_Khal	Slope	Return_Period	q_100

data={ "Code_Name":Code_Name_list,
     "Khal_Name":khal_name_list,
      "Basin_Area":A_list,
      "Length_of_Khal":L_list,
      "Slope":s_list,
      "Return_Period":T_list,
      "q_100":q_100_list,
      "return_period_adjustment" :mf_list,
      "q_10":q_10_list,
      "Basin_Charcteristic_factor":slope_factor_list,
      "Basin_Charcteristic_Correction":cf_list,
      "q_10_adjusted":q_adjusted_list,
      "Basin_Area_Correction":bcf_list,
      "q_10_corrected":q_corrected_list,
      "Q_cusec":Q_cusec_list,
      "Q_cumec":Q_cumec_list,     
     }
Hydrologic_Analysis_Df=pd.DataFrame(data)

ValueError: All arrays must be of the same length

In [ ]:
#Main_Input_Df_fps